In [1]:
import requests
import os

В документации к API image-net предлагается получать ссылки на изображения следующим образом:

```
http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=[wnid]
```

`wnid` — идентификатор категории.

К сожалению, на данный момент накакие изображения (включая кошек и собак) недоступны.

In [2]:
api_url = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid='
wnid_cats = 'n02121808'
wnid_dogs = 'n02084071'

In [3]:
requests.get(api_url + wnid_cats)

<Response [404]>

In [4]:
requests.get(api_url, wnid_dogs)

<Response [404]>

Поэтому изображения будут загружаться с GitHub, но по точно такому же принципу: на отдельных страницах перечислены ссылки на изображения, которые затем используются для загрузки.

In [5]:
BASE_URL = 'https://raw.githubusercontent.com/ulianovkd/test/master/'
TRAIN_RATIO = 0.8

In [6]:
def get_img_urls(category_id):
    '''Returns a list of image urls'''
    response = requests.get(BASE_URL + category_id)
    if response.status_code == 200:
        return response.content.decode('utf-8').split('\n')

In [7]:
def download_image(url, path):
    '''Downloads an image'''
    response = requests.get(url)
    if response.status_code == 200:
        img_data = response.content
        with open(path, 'wb') as file:
            file.write(img_data)

In [8]:
from pymongo import MongoClient
client = MongoClient()
db = client.test_database
col = db.cats_and_dogs

In [9]:
for category in ['cats', 'dogs']:
    img_urls = get_img_urls(category + '.list')
    img_train_size = round(len(img_urls) * TRAIN_RATIO)

    os.makedirs('data/train/' + category, exist_ok=True)
    os.makedirs('data/test/' + category, exist_ok=True)

    for i in range(len(img_urls)):
        if i < img_train_size:
            path = 'data/train/' + category
        else:
            path = 'data/test/' + category

        path += '/img' + str(i) + '.jpg'

        download_image(img_urls[i], path)
        
        doc = {'category': category[:-1],
                'path': path}
        col.insert_one(doc)